In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import pickle as pkl
from eipy.deep_ei import EnsembleIntegration
from scikeras.wrappers import KerasClassifier
import sys

In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
inanimate_labels=[0,1,8,9]
def binarize_array(arr):
    mask = np.isin(arr, inanimate_labels)

    arr = np.where(mask, 0, 1)

    return arr

In [5]:
y_train = binarize_array(y_train)
y_test = binarize_array(y_test)

In [ ]:

with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
    X_train_resized=pkl.load(file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
    X_test_resized=pkl.load(file=file)
file.close()

In [6]:
from keras import layers, models
from keras.applications import ResNet50
res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
averaging_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(averaging_layer)

res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
from skimage.transform import resize
import pickle as pkl
new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')

In [2]:
X_sample = X_train_resized[0:10000]
X_sample.shape

NameError: name 'X_train_resized' is not defined

In [10]:
with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:
    pkl.dump(file=file, obj=X_train_resized)
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:
    pkl.dump(file=file, obj=X_test_resized)

OSError: [Errno 28] No space left on device

In [7]:
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
net_model = Model(inputs=base_model.input, outputs=predictions)

net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
base_predictors = {
    "ResNet50" : res_model,
    "MobileNet" : net_model
}

In [14]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=2,
                        k_inner=2,
                        n_samples=1,
                        sampling_aggregation=None,
                        sampling_strategy=None,
                        n_jobs=1,# -1 could be messing up for one bp
                        metrics=metrics,
                        random_state=38,
                        project_name="images",
                        model_building=True,
                        )

In [15]:
EI.fit_base(X_train_resized, y_train, modality_name="images")

Training base predictors on images...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%

391/391 [==============================] - 33s 84ms/step


Generating ensemble training data: |█████     | 50%

391/391 [==============================] - 34s 87ms/step


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |          |  0%

782/782 [==============================] - 270s 345ms/step - loss: 0.0600 - accuracy: 0.9787


: 

In [1]:
EI.base_summary

NameError: name 'EI' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
ensemble_predictors = {"LR": LogisticRegression(),
                       "XGB": XGBClassifier()}
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

In [ ]:
EI.ensemble_summary

In [ ]:
X_dict = {"images": X_sample}

In [ ]:
EI.modality_names

In [ ]:
EI.predict(X_dict=X_dict, ensemble_model_key="LR")